In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from math import e

In [ ]:

connection = sqlite3.connect('file:database.db?mode=ro', uri=True)

# market_orders = pd.read_sql_query("SELECT * FROM market_orders", connection)
market_history = pd.read_sql_query("SELECT * FROM market_history", connection)
connection.close()

market_history = market_history.sort_values(by=['date'])


In [ ]:
item_34 = market_history.loc[market_history['item_id'] == 29043]
item_34 = item_34.loc[item_34['region_id'] == 10000002]

item_34

In [ ]:
last_30_days = item_34.tail(30)
rolling_7 = item_34['average_price'].rolling(7).mean().tail(30)
rolling_30 = item_34['average_price'].rolling(30).mean().tail(30)

plt.figure()
plt.plot(last_30_days['date'], last_30_days['average_price'], label='average_price', color='orange')
plt.plot(last_30_days['date'], last_30_days['low_price'], label='average_price', color='red')
plt.plot(last_30_days['date'], last_30_days['high_price'], label='average_price', color='green')
plt.plot(last_30_days['date'], rolling_7, label='average_price', color='blue')
plt.plot(last_30_days['date'], rolling_30, label='average_price', color='purple')

plt.fill_between(last_30_days['date'], last_30_days['low_price'], last_30_days['high_price'], alpha=0.2)
plt.xticks(rotation=90)
plt.show()


In [ ]:
last_30_days = item_34.tail(30)
difference = last_30_days['high_price'] - last_30_days['low_price']
relative_to_low = last_30_days['average_price'] - last_30_days['low_price']
relative = relative_to_low / difference
plt.figure()

# plt.plot(last_30_days['date'], difference, label='average_price')
plt.plot(last_30_days['date'], relative, label='average_price')

plt.show()

In [ ]:
item_34['margin'] = ((item_34['high_price'] - item_34['low_price']) / item_34['low_price']) * 100
item_34['expected_history_profit'] = (item_34['high_price'] * 0.91) - (item_34['low_price'] * 1.09)
item_34

## Calculating expected profit value per item

We need to now the open buy and sell prices of the day, via market orders we can calculate the average buy/sell prices of the day.
I think we might also need data of the 7d rolling buy/sell prices.

With this data we can calculate the following useful data points:
- 1d/7d margin
- 1d/7d spread

We use worst case scenario of taxes. 8% taxes on buy and sell prices.
We also have a _variable_ $n$ to describe our buy order frequence which we will use for taxes

$$profit = (sell - 9\%) - (buy + 9\% + (n*1\%))$$

This profit describes the average profit per item.

## Percentage of market capture

From the previous part we have update frequence $n$, which we assume is spread throughout the day.

Now we need to calculate how much of the market we can capture. We have:
- User Update Frequency
- Competition order update frequency.
- Competition order update frequency over time

Based on competition we can calculate the percentage of market capture per day.
$$\text{time_before_competing_order} = (24h / \text{competition_orders})$$
$$\text{time_on_top} = \text{time_before_competing_order} * n$$

## Items bought to Items sold

Items that are bought need to be sold.

We have:
- Volume of items traded
- Volume of buy/sell orders that disapeared
- buy/sell ratio

I guess we can take the lowest of the buy/sell volume averaged over a few days.

In [ ]:
connection = sqlite3.connect('file:database.db?mode=ro', uri=True)

market_orders = pd.read_sql_query("SELECT * FROM market_orders WHERE item_id=2311", connection)
# market_history = pd.read_sql_query("SELECT * FROM market_history", connection)
connection.close()

market_orders = market_orders.sort_values(by=['order_id', 'issued'])

## Useful queries

- Fulfilled: `SELECT order_id, issued, count(*) FROM market_orders GROUP BY order_id, issued HAVING count(*) > 1 ORDER BY order_id, issued LIMIT 100;`
- Updated: `SELECT order_id, count(*) FROM market_orders GROUP BY order_id, volume_remain HAVING count(*) > 1 ORDER BY order_id limit 100;`
- Competition 24 buy `SELECT count(1), item_id FROM market_orders WHERE buy_order=1 AND datetime(issued) >= datetime('now', '-1 Day') GROUP BY item_id;`  
- Competition 24 sell `SELECT count(1), item_id FROM market_orders WHERE buy_order=0 AND datetime(issued) >= datetime('now', '-1 Day') GROUP BY item_id;` 
- Competition 3 buy `SELECT count(1), item_id FROM market_orders WHERE buy_order=1 AND datetime(issued) >= datetime('now', '-3 Hour') GROUP BY item_id;`
- Competition 3 sell `SELECT count(1), item_id FROM market_orders WHERE buy_order=0 AND datetime(issued) >= datetime('now', '-3 Hour') GROUP BY item_id;`

In [ ]:
order_34 = market_orders
order_34.sort_values(by=['order_id', 'issued'])

In [ ]:
order_34 = order_34[order_34.groupby('order_id')['order_id'].transform('size') > 1]
order_34